In [7]:
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

IMG_SIZE = 224
data = []

def create_training_data():
  with open('path.csv','r') as f:
    reader = csv.reader(f)
    for i in reader:
      path = i[0]
      try:
        img_array = cv2.imread(path) # 画像読み込み
        img_resize_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # 画像のリサイズ
        label = i[0][2]
        if label=='A':
          cat = 0
        elif label=='B':
          cat = 1
        elif label=='C':
          cat = 2
        elif label=='D':
          cat = 3
        elif label=='E':
          cat = 4
        data.append([img_resize_array, cat])  # 画像データ、ラベル情報を追加
      except Exception as e:
        pass

create_training_data()

print(len(data))

X = []
y = []

for feature, label in data:
  X.append(feature)
  y.append(label)

X = np.array(X)
y = np.array(y)

Train_image, x_test, Train_label, y_test = train_test_split(X, y, test_size=0.3)
Vali_image, Test_image, Vali_label, Test_label = train_test_split(x_test, y_test, test_size=0.5)

inputs = tf.keras.Input(shape=(None, None, 3))
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (224, 224)))(inputs)
x = tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)(x)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights='imagenet', input_tensor=x, input_shape=(224, 224, 3),
    include_top=False, pooling='avg'
)

base_model.trainable = True

model = tf.keras.Sequential()
model.add(base_model)
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(Train_image, Train_label, epochs=50, validation_data=(Vali_image, Vali_label), batch_size=10)

print(model.evaluate(Test_image, Test_label, verbose=0))

hist = history.history
x_arr = np.arange(len(hist['loss'])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)

#plt.savefig('figures/15_18.png', dpi=300)
plt.show()


784
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense_2 (Dense)             (None, 256)               327936    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                  

KeyboardInterrupt: 